In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg
robot = Robot()

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [2]:
####CREATE A DATASET DIRECTORY####
DATASET_DIR = 'dataset_xy'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directory already exist')

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x*50+50, y*50+50, uuid1())

motor_speed_ratio = 0.3
time_interval = 0.1

####BUTTON FUNCTION####
def stop(change):
    robot.stop()

def step_forward(change):
    robot.forward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
def step_backward(change):
    robot.backward(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
def step_left(change):
    robot.left(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()

def step_right(change):
    robot.right(motor_speed_ratio)
    time.sleep(time_interval)
    robot.stop()
    
def save_snapshot(change):
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

####CAMERA SETTINGS####
camera = Camera.instance(width=224, height=224, fps=5)
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jped', width=224, height=224)

####SELF-DEFINED LAYOUT####
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')

stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button])

save_button = widgets.Button(description='save', layout=button_layout)
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')
slider_box = widgets.VBox([x_slider, y_slider, count_widget, save_button])

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112,224), 8, (0, 0, 255), 3)
    image = cv2.line(image, (x, y), (112, 224), (255, 0, 0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

####LINK BUTTONS TO ACTIONS####
save_button.on_click(save_snapshot)
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
###############################################

display(widgets.HBox([target_widget, slider_box, controls_box]))

Directory already exist
